In [1]:
import pandas as pd 
from sklearn.preprocessing import LabelEncoder
import sys
import anndata as an
import scanpy as sp
import h5py

sys.path.append('../src')
sys.path.append('../tests')

from models.lib.lightning_train import DataModule, generate_trainer
from models.lib.data import *
from models.lib.neural import *
from models.lib.testing import *

from pytorch_lightning.loggers import WandbLogger
from torchmetrics.functional import *

import pandas as pd
from scipy.sparse import csr_matrix
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

from bigcsv.bigcsv import experimental_to_h5ad
from functools import partial
import torchmetrics.functional as f
from torchmetrics import Metric
import torchmetrics 

In [2]:
data = an.read_h5ad('../data/pancreas/pancreas.h5ad')
data

/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/anndata/compat/__init__.py:180: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.
  warn(
/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/anndata/compat/__init__.py:180: FutureWarning: Moving element from .uns['neighbors']['connectivities'] to .obsp['connectivities'].

This is where adjacency matrices should go now.
  warn(


AnnData object with n_obs × n_vars = 14693 × 2448
    obs: 'celltype', 'sample', 'n_genes', 'batch', 'n_counts', 'louvain'
    var: 'n_cells-0', 'n_cells-1', 'n_cells-2', 'n_cells-3'
    uns: 'celltype_colors', 'louvain', 'neighbors', 'pca', 'sample_colors'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    obsp: 'distances', 'connectivities'

In [7]:
vals = data.obs.groupby('batch')['celltype'].unique()
vals

batch
0    ['acinar', 'beta', 'delta', 'activated_stellat...
1    ['alpha', 'ductal', 'endothelial', 'delta', 'a...
2    ['not applicable', 'delta', 'alpha', 'gamma', ...
3    ['dropped', 'alpha', 'ductal', 'delta', 'beta'...
Name: celltype, dtype: object

In [8]:
vals[0]

['acinar', 'beta', 'delta', 'activated_stellate', 'ductal', ..., 'quiescent_stellate', 'macrophage', 'schwann', 'mast', 't_cell']
Length: 14
Categories (24, object): ['MHC class II', 'PSC', 'acinar', 'activated_stellate', ..., 't_cell', 'unclassified', 'unclassified endocrine', 'unclear']

In [9]:
vals[1]

['alpha', 'ductal', 'endothelial', 'delta', 'acinar', 'beta', 'unclear', 'gamma', 'mesenchymal', 'epsilon']
Categories (24, object): ['MHC class II', 'PSC', 'acinar', 'activated_stellate', ..., 't_cell', 'unclassified', 'unclassified endocrine', 'unclear']

In [16]:
set(vals.values)

TypeError: unhashable type: 'Categorical'

In [15]:
vals.values

array([['acinar', 'beta', 'delta', 'activated_stellate', 'ductal', ..., 'quiescent_stellate', 'macrophage', 'schwann', 'mast', 't_cell']
       Length: 14
       Categories (24, object): ['MHC class II', 'PSC', 'acinar', 'activated_stellate', ..., 't_cell', 'unclassified', 'unclassified endocrine', 'unclear'],
       ['alpha', 'ductal', 'endothelial', 'delta', 'acinar', 'beta', 'unclear', 'gamma', 'mesenchymal', 'epsilon']
       Categories (24, object): ['MHC class II', 'PSC', 'acinar', 'activated_stellate', ..., 't_cell', 'unclassified', 'unclassified endocrine', 'unclear'],
       ['not applicable', 'delta', 'alpha', 'gamma', 'ductal', ..., 'PSC', 'endothelial', 'epsilon', 'mast', 'unclassified']
       Length: 15
       Categories (24, object): ['MHC class II', 'PSC', 'acinar', 'activated_stellate', ..., 't_cell', 'unclassified', 'unclassified endocrine', 'unclear'],
       ['dropped', 'alpha', 'ductal', 'delta', 'beta', 'gamma', 'acinar', 'mesenchyme']
       Categories (24, objec

pandas.core.arrays.categorical.Categorical